[Dataset](https://www.kaggle.com/arashnic/building-sites-power-consumption-dataset)

In [1]:
#!kaggle datasets download -d arashnic/building-sites-power-consumption-dataset
#!unzip building-sites-power-consumption-dataset.zip -d ./dataset

# Implementation

## Packages

In [2]:
# data elaboration functions
import pandas as pd
from six.moves import collections_abc
import string
import numpy as np

# datetime functions
import datetime as dt

# file management functions
import os
import sys
import opendatasets as od
import pickle
from pathlib import Path

# plot functions
import matplotlib.pyplot as plt
%matplotlib inline

# data science functions
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
import joblib
from sklearn.metrics import mean_absolute_error

# configuration file
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from Configuration.config import cfg_path

# custom functions
from Code.Plotting.plots import Plots
from Code.Regressors.regressors import Regressors
from Code.Scoring.scoring import Scoring
from Code.Scoring.train_test import TrainTest
from Code.Scoring.train import Training
from Code.Scoring.forecast import Forecasting
from Code.Scoring.kpi import Kpi
from Code.Scoring.scoring import Scoring
from Code.Utils.utils import Utils


## Setup

In [3]:
#od.download("https://www.kaggle.com/arashnic/building-sites-power-consumption-dataset/download")
root = Path(os.getcwd()).parent
dataset_path = os.path.join(root, cfg_path.data_dir.input_path)

## Load Data


In [4]:
df_holidays = pd.read_csv(os.path.join(dataset_path, "power-laws-forecasting-energy-consumption-holidays.csv"), sep=";", parse_dates=["Date"])
df_holidays = Utils.columns_camel_to_snake(df_holidays)
print("Holiday columns: ", list(df_holidays.columns))
print("NaNs:", df_holidays.isna().sum().values.sum())

df_metadata = pd.read_csv(os.path.join(dataset_path, "power-laws-forecasting-energy-consumption-metadata.csv"), sep=";")
df_metadata = Utils.columns_camel_to_snake(df_metadata)
print("Metadata columns: ", list(df_metadata.columns))
print("NaNs:", df_metadata.isna().sum().values.sum())

df_weather = pd.read_csv(os.path.join(dataset_path, "power-laws-forecasting-energy-consumption-weather.csv"), sep=";", parse_dates=["Timestamp"])
df_weather = Utils.columns_camel_to_snake(df_weather)
print("Weather columns: ", list(df_weather.columns))
print("NaNs:", df_weather.isna().sum().values.sum())

df_train_data = pd.read_csv(os.path.join(dataset_path, "power-laws-forecasting-energy-consumption-training-data.csv"), sep=";", parse_dates=["Timestamp"])
df_train_data = Utils.columns_camel_to_snake(df_train_data)
print("Train columns: ", list(df_train_data.columns))
print("NaNs:", df_train_data.isna().sum().values.sum())

Holiday columns:  ['date', 'holiday', 'site_id']
NaNs: 0
Metadata columns:  ['site_id', 'surface', 'sampling', 'base_temperature', 'monday_is_day_off', 'tuesday_is_day_off', 'wednesday_is_day_off', 'thursday_is_day_off', 'friday_is_day_off', 'saturday_is_day_off', 'sunday_is_day_off']
NaNs: 0
Weather columns:  ['timestamp', 'temperature', 'distance', 'site_id']
NaNs: 0
Train columns:  ['obs_id', 'site_id', 'timestamp', 'forecast_id', 'value']
NaNs: 86601


# Data Preparation


## Parameter setup

In [5]:
id = 'site_id'
list_unique_id = ['site_id', 'timestamp']
list_temp = ['temp']
y = 'value'

## Dealing with NAs

#### Create a full time sequence on a chosen frequency

In [6]:
# df_train_data
df_train_data.head()
print('List ids:', list(df_train_data[id].unique()))

List ids: [42, 43, 40, 41, 63, 66, 67, 68, 64, 65, 69, 70, 72, 73, 115, 119, 116, 117, 118, 59, 60, 62, 61, 112, 109, 110, 111, 8, 9, 25, 26, 46, 47, 48, 49, 106, 107, 100, 101, 102, 105, 50, 51, 52, 53, 108, 14, 10, 11, 12, 13, 22, 23, 57, 54, 58, 93, 94, 96, 98, 99, 92, 44, 45, 88, 87, 89, 90, 84, 85, 86, 78, 83, 228, 222, 223, 224, 225, 226, 227, 215, 229, 230, 273, 276, 271, 274, 275, 272, 74, 77, 76, 75, 218, 219, 221, 233, 234, 269, 270, 235, 236, 237, 216, 217, 204, 209, 261, 262, 263, 264, 265, 266, 267, 268, 232, 212, 213, 205, 206, 207, 208, 210, 211, 260, 259, 255, 256, 257, 231, 252, 254, 250, 251, 253, 238, 239, 240, 241, 243, 244, 245, 246, 247, 248, 249, 123, 124, 125, 126, 121, 122, 120, 127, 128, 129, 200, 201, 199, 141, 140, 135, 136, 139, 202, 203, 151, 149, 152, 146, 148, 150, 190, 191, 19, 167, 162, 173, 171, 172, 169, 170, 163, 164, 165, 143, 145, 142, 197, 194, 195, 196, 192, 193, 160, 161, 153, 154, 155, 156, 157, 158, 159, 198, 175, 174, 183, 184, 185, 186, 189

In [7]:
date_var = Utils.find_date(df_train_data)
freq = dt.timedelta(days=1)
dict_grouping = {'obs_id': 'first', 'forecast_id': 'first', 'value': 'sum'}
df_resampled = Utils.resample_data(df_train_data, id, date_var, freq, dict_grouping)
print('List ids after resampling:', list(df_resampled[id].unique()))

find_date, date_col found: ['timestamp']
resample_data: variable obs_id
resample_data: variable obs_id completed
resample_data: variable forecast_id
resample_data: variable forecast_id completed
resample_data: variable value
resample_data: variable value completed
                       timestamp     obs_id  site_id  forecast_id  \
0      2015-11-02 00:00:00+00:00  3187477.0       42       1080.0   
1      2015-11-03 00:00:00+00:00  1309542.0       42       1080.0   
2      2015-11-04 00:00:00+00:00  6998589.0       42       1080.0   
3      2015-11-05 00:00:00+00:00  5708558.0       42       1080.0   
4      2015-11-06 00:00:00+00:00  6931527.0       42       1080.0   
...                          ...        ...      ...          ...   
188662 2015-02-12 00:00:00+00:00   954528.0       34        981.0   
188663 2015-02-13 00:00:00+00:00  6319073.0       34        981.0   
188664 2015-02-14 00:00:00+00:00   849485.0       34        981.0   
188665 2015-02-15 00:00:00+00:00   141866.0  

In [8]:
df_train_data = Utils.add_seq(df_resampled, date_var, serie = id, freq = freq, end_date='', start_date='')
df_train_data.head()
print('List ids after resampling and adding full time sequence:', list(df_train_data[id].unique()))

Adding sequence to serie 42 as 1 of 267
Adding sequence to serie 43 as 2 of 267
Adding sequence to serie 40 as 3 of 267
Adding sequence to serie 41 as 4 of 267
Adding sequence to serie 63 as 5 of 267
Adding sequence to serie 66 as 6 of 267
Adding sequence to serie 67 as 7 of 267
Adding sequence to serie 68 as 8 of 267
Adding sequence to serie 64 as 9 of 267
Adding sequence to serie 65 as 10 of 267
Adding sequence to serie 69 as 11 of 267
Adding sequence to serie 70 as 12 of 267
Adding sequence to serie 72 as 13 of 267
Adding sequence to serie 73 as 14 of 267
Adding sequence to serie 115 as 15 of 267
Adding sequence to serie 119 as 16 of 267
Adding sequence to serie 116 as 17 of 267
Adding sequence to serie 117 as 18 of 267
Adding sequence to serie 118 as 19 of 267
Adding sequence to serie 59 as 20 of 267
Adding sequence to serie 60 as 21 of 267
Adding sequence to serie 62 as 22 of 267
Adding sequence to serie 61 as 23 of 267
Adding sequence to serie 112 as 24 of 267
Adding sequence to 

In [9]:
# df_holidays
df_holidays.head()

,date,holiday,site_id
0,2016-02-15,Washington's Birthday,1
1,2017-05-29,Memorial Day,1
2,2017-11-23,Thanksgiving Day,1
3,2017-12-29,New Years Eve Shift,1
4,2017-12-31,New Years Eve,1


In [10]:
date_var = Utils.find_date(df_holidays)
freq = dt.timedelta(days=1)
dict_grouping = {'holiday': 'first'}
df_resampled = Utils.resample_data(df_holidays, id, date_var, freq, dict_grouping)

find_date, date_col found: ['date']
resample_data: variable holiday
resample_data: variable holiday completed
             date        holiday  site_id
0      2014-01-01       New year        1
1      2014-01-02           None        1
2      2014-01-03           None        1
3      2014-01-04           None        1
4      2014-01-05           None        1
...           ...            ...      ...
257126 2017-12-21           None      288
257127 2017-12-22           None      288
257128 2017-12-23           None      288
257129 2017-12-24           None      288
257130 2017-12-25  Christmas Day      288

[257131 rows x 3 columns]


In [11]:
# Regressor dataset do not need a full time sequence to check for NAs
df_holidays = df_resampled.copy()
df_holidays.head()

,date,holiday,site_id
0,2014-01-01,New year,1
1,2014-01-02,None,1
2,2014-01-03,None,1
3,2014-01-04,None,1
4,2014-01-05,None,1


In [12]:
# df_metadata 
# does not contain a time series, therefore it cannot be resampled and do not need a full time sequence to check for NAs
df_metadata.head()

,site_id,surface,sampling,base_temperature,monday_is_day_off,tuesday_is_day_off,wednesday_is_day_off,thursday_is_day_off,friday_is_day_off,saturday_is_day_off,sunday_is_day_off
0,207,7964.873347,30.0,18.0,False,False,False,False,False,True,True
1,7,15168.125971,30.0,18.0,False,False,False,False,False,True,True
2,74,424.340663,15.0,18.0,False,False,False,False,False,True,True
3,239,1164.822636,15.0,18.0,False,False,False,False,False,True,True
4,274,1468.246690,5.0,18.0,False,False,False,False,False,True,True


In [13]:
# df_weather
df_weather.head()

,timestamp,temperature,distance,site_id
0,2017-03-03 19:00:00+00:00,10.6,27.489346,51
1,2017-03-03 19:20:00+00:00,11.0,28.663082,51
2,2017-03-03 20:00:00+00:00,6.3,28.307039,51
3,2017-03-03 21:55:00+00:00,10.0,29.797449,51
4,2017-03-03 23:00:00+00:00,5.4,28.307039,51


In [14]:
date_var = Utils.find_date(df_weather)
freq = dt.timedelta(days=1)
dict_grouping = {'temperature': 'mean', 'distance': 'mean'}
df_resampled = Utils.resample_data(df_weather, id, date_var, freq, dict_grouping)

find_date, date_col found: ['timestamp']
resample_data: variable temperature
resample_data: variable temperature completed
resample_data: variable distance
resample_data: variable distance completed
                      timestamp  temperature  site_id   distance
0     2015-01-01 00:00:00+00:00    -0.038168       51  28.196452
1     2015-01-02 00:00:00+00:00     1.806015       51  28.194644
2     2015-01-03 00:00:00+00:00     3.314179       51  28.102067
3     2015-01-04 00:00:00+00:00     4.661654       51  28.089319
4     2015-01-05 00:00:00+00:00     5.380741       51  28.615182
...                         ...          ...      ...        ...
61557 2017-12-09 00:00:00+00:00          NaN       13        NaN
61558 2017-12-10 00:00:00+00:00     4.650000       13  25.252279
61559 2017-12-11 00:00:00+00:00     9.250000       13  24.200407
61560 2017-12-12 00:00:00+00:00     7.650000       13  25.252279
61561 2017-12-13 00:00:00+00:00     5.000000       13  24.200407

[61562 rows x 4 colu

In [15]:
# Regressor dataset do not need a full time sequence to check for NAs
df_weather = df_resampled.copy()
df_weather.head()

,timestamp,temperature,site_id,distance
0,2015-01-01 00:00:00+00:00,-0.038168,51,28.196452
1,2015-01-02 00:00:00+00:00,1.806015,51,28.194644
2,2015-01-03 00:00:00+00:00,3.314179,51,28.102067
3,2015-01-04 00:00:00+00:00,4.661654,51,28.089319
4,2015-01-05 00:00:00+00:00,5.380741,51,28.615182


#### Plotting y series

In [16]:
count = 1
for i in list(df_train_data[id].unique()):
    print('Plotting id:', i, 'as', count, 'of', len(list(df_train_data[id].unique())))
    plot = Plots.sliding_line_plot(df_train_data, y, id, i, chart_title="")
    plot.write_html(os.path.join(root, cfg_path.data_dir.plot_path, id + '_' + str(i) + ".html"))
    count = count + 1 

Plotting id: 42 as 1 of 267
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 42
Plotting id: 43 as 2 of 267
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 43
Plotting id: 40 as 3 of 267
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 40
Plotting id: 41 as 4 of 267
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 41
Plotting id: 63 as 5 of 267
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 63
Plotting id: 66 as 6 of 267
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 66
Plotting id: 67 as 7 of 267
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 67
Plotting id: 68 as 8 of 267
find_date, date_col found: ['timestamp']
sliding_line_plot: plotting Value site_id 68
Plotting id: 64 as 9 of 267
find_date, date_col found: ['timestamp']
sliding_line_plot: 

## Creating working dataset

In [17]:
# Site Ids available in all data dfs
common_site_ids = list(set(df_holidays[id].unique()) & set(df_weather[id].unique()) & set(df_train_data[id].unique()))

# Final df
df_final = df_train_data[df_train_data[id].isin(common_site_ids)].copy()

# Date
date_var = Utils.find_date(df_final)

find_date, date_col found: ['timestamp']


#### Count NAs in y by id

In [18]:
df_final.head()
print('List ids in df_final:', list(df_final[id].unique()))

List ids in df_final: [42, 40, 25, 26, 46, 47, 48, 49, 50, 51, 52, 12, 13, 22, 23, 57, 44, 45, 16, 20, 21, 17, 18, 27, 33, 29, 32, 39, 34]


In [19]:
pivotna = pd.pivot_table(df_final[df_final[y].isna()], index=id, values = y, aggfunc='count').reset_index()
pivotna.rename(columns={y: y + '_count_NA'})
pivotna

,site_id,value
0,12,0
1,13,0
2,16,0
3,17,0
4,18,0
5,20,0
6,21,0
7,22,0
8,23,0
9,25,0


### Adding regressors

#### Holidays

In [20]:
df_final = Regressors.merge_holidays_by_date(df_final, df_holidays, id)

find_date, date_col found: ['timestamp']
find_date, date_col found: ['date']


#### Site leaves

In [21]:
dict_days_off = {'friday_is_day_off': 5, 'saturday_is_day_off': 6, 'sunday_is_day_off': 7}
df_final = Regressors.merge_additional_days_off(df_final, df_metadata, id, dict_days_off)

find_date, date_col found: ['timestamp']


#### Additional metadata

In [22]:
df_final = pd.merge(df_final, df_metadata[[id, "surface", "base_temperature"]], how="left", on=[id], validate = 'm:1')

#### Other calendar variables

In [23]:
df_final = Regressors.add_weekdays(df_final, date_var)
df_final = Regressors.add_months(df_final, date_var)

#### Weather

In [24]:
df_final = Regressors.merge_weather(df_final, df_weather, date_var, id)

In [25]:
df_final = Regressors.calculate_degree_days(df_final, base_temperature = "base_temperature", temperature = "temperature")
df_final.columns

Index(['site_id', 'timestamp', 'obs_id', 'forecast_id', 'value', 'holidays',
       'day_off', 'surface', 'base_temperature', 'wd_mon', 'wd_tue', 'wd_wed',
       'wd_thu', 'wd_fri', 'wd_sat', 'wd_sun', 'month_01', 'month_02',
       'month_03', 'month_04', 'month_05', 'month_06', 'month_07', 'month_08',
       'month_09', 'month_10', 'month_11', 'month_12', 'temperature',
       'distance', 'DDC_temperature', 'DDH_temperature'],
      dtype='object')

#### Remove NAs and duplicates

In [26]:
df_final = df_final.dropna()
print('List ids in df_final after removing NAs:', list(df_final[id].unique()))
assert df_final[df_final[y].isna()].count().sum() == 0, "y should not contain NaNs"

List ids in df_final after removing NAs: [13, 16, 21, 22, 25, 26, 29, 32, 33, 39, 40, 42, 27, 51, 47, 48, 49, 44, 50, 57, 20, 46, 17, 18, 52, 12]


In [27]:
df_final = df_final.drop_duplicates()
print('List ids in df_final after removing duplicates:', list(df_final[id].unique()))
assert df_final[df_final.duplicated()].count().sum() == 0, "y should not contain duplicates"

List ids in df_final after removing duplicates: [13, 16, 21, 22, 25, 26, 29, 32, 33, 39, 40, 42, 27, 51, 47, 48, 49, 44, 50, 57, 20, 46, 17, 18, 52, 12]


# Saving

In [28]:
df_final.to_pickle(os.path.join(root, cfg_path.data_dir.output_path, 'df_final.pkl'))